In [ ]:
import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import dash_table as dt
import pandas as pd
import base64
from dash.dependencies import Input, Output
# the class that contains login and CRUD methods
from AnimalShelter import AnimalShelter


app = dash.Dash(__name__)
application = app.server
app.title = 'Dashboard'


###########################
# Data Manipulation / Model
###########################


shelter = AnimalShelter()

# class read method must support return of cursor object and accept projection json input
search = {}
data = shelter.read(search)

# used for debug purposes
# print (str(data))

df = pd.DataFrame.from_records(data, columns=['id', 'age_upon_outcome', 'animal_id', 'animal_type',
                                              'breed', 'color','date_of_birth', 'datetime', 'monthyear',
                                              'name', 'outcome_subtype','outcome_type', 'sex_upon_outcome',
                                              'location_lat', 'location_long','age_upon_outcome_in_weeks'])

#########################
# Dashboard Layout / View
#########################
# loads company's logo name in the vairable
company_logo = 'logo-image.png'
encoded_pic_logo = base64.b64encode(open(company_logo, 'rb').read())

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display': 'none'}),
    # used www.snhu.edu as the hperlink over the logo, per Grazioso Salvare's request
    html.A(href="http://www.snhu.edu", children=html.Img(src='data:image/png;base64,{}'.format(encoded_pic_logo.decode()),
                                                                     style={'width': '80px', 'height': '80px'})),
    html.Center(html.B(html.H1('Grazioso Salvare - By Mahdi B.'))),
    html.Hr(),

    # Drop down display animal by type
    # creates a drop-down menu enabling user to select animal based on different types
    html.Div(className='row',
             style={'color': 'darkbrown', 'font-size': 15, 'margin-righ': '80px'},
             children=[dcc.RadioItems(id='input-radio-button', options=[
                 {"label": "Reset .  .", "value": "All"},
                 {"label": "Water Rescue .  .", "value": "WR"},
                 {"label": "Mountain or Wilderness .  .", "value": "MoW"},
                 {"label": "Disaster or Individual Tracking .  .", "value": "DoIT"}
             ],
                                    value='All',
                                    inline=True
                                    )
                       ]),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        # Set up the features for your interactive data table to make it user-friendly for your client
        editable=False,
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        filter_action="native",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=8,
    ),
    html.Br(),
    html.Hr(),
    # This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
             style={'display': 'flex'},
             children=[
                 html.Div(
                     id='graph-id',
                     className='col s12 m6',
                 ),
                 html.Div(
                     id='map-id',
                     className='col s12 m6',
                 )
             ])
])

#############################################
# Interaction Between Components / Controller
#############################################
# Dropdown of animal types or all
@app.callback(
    Output('datatable-id', 'data'),
    [Input('input-radio-button', 'value')])
def on_click(radio_value):
    global df
    global data
    # start case
    # WR stands for water rescure
    if radio_value == "WR":
        data = shelter.read({"animal_type": "Dog", "sex_upon_outcome": "Intact Female",
                             "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
                             "age_upon_outcome_in_weeks":{"$lt":156, "$gt":26}})
    # MoW stands for Mountain or Wilderness 
    elif radio_value == "MoW":
        data = shelter.read({"animal_type": "Dog", "sex_upon_outcome": "Intact Male",
                             "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
                             "age_upon_outcome_in_weeks":{"$lt":156, "$gt":26}})
    # DoIT stands for Mountain or Wilderness                         
    elif radio_value == "DoIT":
        data = shelter.read({"animal_type": "Dog", "sex_upon_outcome": "Intact Male",
                             "breed": {"$in": ["German Shepherd","Doberman Pinscher","Golden Retriever", "Bloodhound", "Rottweiler"]},
                             "age_upon_outcome_in_weeks":{"$lt":300, "$gt":20}})
    elif radio_value == "All":
        data = shelter.read({})
    df = pd.DataFrame.from_records(data,
                                   columns=['id', 'age_upon_outcome', 'animal_id', 'animal_type', 'breed', 'color',
                                            'date_of_birth', 'datetime', 'monthyear', 'name', 'outcome_subtype',
                                            'outcome_type', 'sex_upon_outcome', 'location_lat', 'location_long',
                                            'age_upon_outcome_in_weeks'])
    return df.to_dict('records')


# pie chart based on animal types
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(show_query):

    dff = pd.DataFrame.from_dict(show_query)
    breeds = []
    for i in range(0, dff.__len__() - 1):
        breeds.append(str(dff.iloc[i, 4]))
    return [
        dcc.Graph(
            figure={
                'data': [
                    {
                        'labels': breeds,
                        'type': 'pie'
                    }
                ],
                'layout': {
                    'title': 'Breeds',
                }
            }
        )
    ]


# location on the map is showed based on the first row
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(show_query):
    if show_query is None:
        return []
    else:
        # Add in the code for your geolocation chart
        dff = pd.DataFrame.from_dict(show_query)
        # Overland Park, KS is located at Latitude: 38.984764 Longitude: -94.677658
        return [
            dl.Map(style={'width': '900px', 'height': '450px'}, center=[dff.iloc[0, 13], dff.iloc[0, 14]], zoom=10,
                   children=[
                       dl.TileLayer(id="base-layer-id"),

                       dl.Marker(position=[dff.iloc[0, 13], dff.iloc[0, 14]], children=[
                           dl.Tooltip(dff.iloc[0, 4]),
                           dl.Popup([
                               html.H1(dff.iloc[0, 9]),
                               html.P(dff.iloc[0, 8])
                           ])
                       ])
                   ])
        ]


if __name__ == '__main__':
    application.run(port=9000)


/var/folders/24/32lrj18x3b11svb5vz24sklh0000gn/T/ipykernel_50087/136492588.py:3: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
/var/folders/24/32lrj18x3b11svb5vz24sklh0000gn/T/ipykernel_50087/136492588.py:4: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
/var/folders/24/32lrj18x3b11svb5vz24sklh0000gn/T/ipykernel_50087/136492588.py:5: UserWarning: 
The dash_table package is deprecated. Please replace
`import dash_table` with `from dash import dash_table`

Also, if you're using any of the table format helpers (e.g. Group), replace 
`from dash_table.Format import Group` with 
`from dash.dash_table.Format import Group`
  import dash_table as dt


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:9000
Press CTRL+C to quit
127.0.0.1 - - [23/Jun/2023 19:34:33] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Jun/2023 19:34:33] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [23/Jun/2023 19:34:33] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [23/Jun/2023 19:34:33] "GET /_dash-component-suites/dash/dash_table/async-highlight.js HTTP/1.1" 304 -
127.0.0.1 - - [23/Jun/2023 19:34:33] "GET /_dash-component-suites/dash/dash_table/async-table.js HTTP/1.1" 304 -
127.0.0.1 - - [23/Jun/2023 19:34:33] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Jun/2023 19:34:33] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Jun/2023 19:34:33] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [23/Jun/2023 19:34:33] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [23/Jun/2023 19:34:33] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Jun/2023 19:34:33] "P